In [ ]:
pip install eli5

     |████████████████████████████████| 106 kB 40.9 MB/s 


In [ ]:
pip install pdpbox

     |████████████████████████████████| 34.0 MB 41 kB/s 
     |████████████████████████████████| 13.1 MB 6.6 kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.1-py3-none-any.whl size=35758225 sha256=5f0ccffdfd62ceb989b8a470d7158fa4962aa3237c88dc92c03cc21c82c25a46
  Stored in directory: /root/.cache/pip/wheels/f4/d0/1a/b80035625c53131f52906a6fc4dd690d8efd2bf8af6a4015eb
Successfully built pdpbox
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
#Importing all the necessary Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import eli5

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn import metrics
from sklearn.metrics import classification_report
from pdpbox import pdp, info_plots

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Loading training and test data
train_data = pd.read_csv('/content/drive/MyDrive/Customer Transaction Prediction/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Customer Transaction Prediction/test.csv')

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
# Function for finding missing values and datatypes of the features

def missing_data(data):
    missing = data.isnull().sum()
    df = pd.concat([missing], axis =1 , keys = ['Missing Values'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    df['Types'] = types
    return(np.transpose(df))

In [ ]:
missing_data(train_data)

In [ ]:
missing_data(test_data)

In [ ]:
sns.countplot(train_data['target'])

It can be seen that out data is imbalanced.

In [ ]:
print("{}% target values with 1". format(100 * train_data["target"].value_counts()[1]/train_data.shape[0]))

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))
train_data['target'].value_counts().plot.pie(explode=[0,0.1],autopct = '%1.1f%%', ax = ax[0], shadow = True)
ax[0].set_title('target')
ax[0].set_ylabel('')
sns.countplot('target', data=train_data,ax=ax[1])
ax[1].set_title('target')
plt.show()

In [ ]:
train_data['target'].value_counts()

In [ ]:
print("Skewness: %f " % train_data['target'].skew())
print("Kurtosis: %f" % train_data['target'].kurt())

Duplicate Values

In [ ]:
train_features = train_data.columns.values[2:202]
test_features = test_data.columns.values[1:201]

In [ ]:
unique_train = []
unique_test = []
for feature in train_features:
    train_values = train_data[feature].value_counts()
    unique_train.append([feature, train_values.max(), train_values.idxmax()])
for feature in test_features:
    test_values = test_data[feature].value_counts()
    unique_test.append([feature, test_values.max(), test_values.idxmax()])

In [ ]:
np.transpose((pd.DataFrame(unique_train, columns = ['Feature', 'Max Duplicates', 'Value']))
             .sort_values(by = 'Max Duplicates', ascending = False))

In [ ]:
np.transpose((pd.DataFrame(unique_test, columns = ['Feature', 'Max Duplicates', 'Value']))
             .sort_values(by = 'Max Duplicates', ascending = False))

Feature Engineering

In [ ]:
for df in [train_data]:
    df['sum'] = df[train_features].sum(axis=1)
    df['min'] = df[train_features].min(axis=1)
    df['max'] = df[train_features].max(axis=1)
    df['mean'] = df[train_features].mean(axis=1)
    df['std'] = df[train_features].std(axis=1)
    df['skew'] = df[train_features].skew(axis=1)
    df['kurt'] = df[train_features].kurtosis(axis=1)
    df['median'] = df[train_features].median(axis=1)

In [ ]:
train_data[train_data.columns[202:]].head(10)

In [ ]:
for df in [test_data]:
    df['sum'] = df[test_features].sum(axis=1)
    df['min'] = df[test_features].min(axis=1)
    df['max'] = df[test_features].max(axis=1)
    df['mean'] = df[test_features].mean(axis=1)
    df['std'] = df[test_features].std(axis=1)
    df['skew'] = df[test_features].skew(axis=1)
    df['kurt'] = df[test_features].kurtosis(axis=1)
    df['median'] = df[test_features].median(axis=1)

In [ ]:
test_data[test_data.columns[201:]].head(10)

Models

In [ ]:
cols = ['target', 'ID_code', 'sum', 'min', 'max', 'mean', 'std', 'skew', 'kurt', 'median']
X = train_data.drop(cols,axis=1)
y = train_data['target']

In [ ]:
X_test = test_data.drop("ID_code", axis = 1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X,y, random_state = 1)

In [ ]:
logistic_clf = LogisticRegression().fit(x_train,y_train)

In [ ]:
preds = logistic_clf.predict(x_val)

In [ ]:
metrics.mean_absolute_error(y_val, preds)

In [ ]:
metrics.mean_squared_error(y_val, preds)

In [ ]:
print(classification_report(y_val, preds))

In [ ]:
from eli5.sklearn import PermutationImportance

In [ ]:
perm = PermutationImportance(logistic_clf, random_state=1).fit(x_val,y_val)

In [ ]:
eli5.show_weights(perm, feature_names = x_val.columns.tolist(), top = 200)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X,y, random_state = 1)

In [ ]:
tree_clf = DecisionTreeClassifier().fit(x_train,y_train)


In [ ]:
cols = ['sum', 'min', 'max', 'mean', 'std', 'skew', 'kurt', 'median']
train_data = train_data.drop(cols, axis=1)

In [ ]:
features = [i for i in train_data.columns if i not in ['ID_code', 'target']]

In [ ]:
tree_graph = tree.export_graphviz(tree_clf, feature_names = features)

In [ ]:
graphviz.Source(tree_graph)

In [ ]:
pdp_goals = pdp.pdp_isolate(model=tree_clf, dataset=x_val, model_features=features, feature = 'var_81' )

# plot it
pdp.pdp_plot(pdp_goals, 'var_81')
plt.show()

In [ ]:
pdp_goals = pdp.pdp_isolate(model=tree_clf, dataset=x_val, model_features=features, feature = 'var_81' )

# plot it
pdp.pdp_plot(pdp_goals, 'var_68')
plt.show()

In [ ]:
row_to_show = 5
data_for_prediction = x_val.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)


tree_clf.predict_proba(data_for_prediction_array)

In [ ]:
pip install shap

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(tree_clf, )

# Calculate Shap values
shap_values = explainer.shap_values(x_val, check_additivity=False )



shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], x_val)



In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(tree_clf, )

# Calculate Shap values
shap_values = explainer.shap_values(x_val, check_additivity=False )


shap.summary_plot(shap_values[1], x_val)